In [1]:
from DDPG.ddpg import DDPG
from DDPG.noise import OrnsteinUhlenbeckActionNoise, NormalNoise
from DDPG.replay_memory import ReplayMemory, Transition

from Simulator.Constants import WIN_THRESHOLD, IMG_HEIGHT, IMG_WIDTH, NUM_FRAMES_STACKED
from DDPG.DriveEnv import DriveEnv

import torch
import numpy as np

In [2]:
checkpoint_file =  "saved_models_converge_1/ep_1800.pth.tar"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = DriveEnv()
agent = DDPG(0,
             0,
             [400, 300],
             IMG_HEIGHT * IMG_WIDTH * NUM_FRAMES_STACKED,
             env.action_space,
             )

agent.load_checkpoint(checkpoint_file)

Saving all checkpoints to ./saved_models/


OSError: Checkpoint not found

In [ ]:
def evaluate():
    agent.actor.eval()
    state = env.reset()
    memory = env.runEpisode(agent, None)
    return memory

In [ ]:
memory = evaluate()
poses = [(i[0][3], i[0][4]) for i in env.controllerRecords]

#normalize poses to start at 0,0
# poses = [(i[0] - poses[0][0], i[1] - poses[0][1]) for i in poses]
currents = [(i[0][5], i[0][6]) for i in env.controllerRecords]
voltages = []

frames, rewards, next_frames, dones = [], [], [], []
for transition in memory:
    state, action, reward, next_state, done = transition
    frames.append(state.numpy()[3, :, :])
    rewards.append(reward)
    next_frames.append(next_state)
    dones.append(done)
    voltages.append(action.cpu().numpy())

#trim poses to length of frames
poses = poses[:len(frames)]
currents = currents[:len(frames)]

#convert voltages to range -12 to 12
voltages = [(i[0] * 12, i[1] * 12) for i in voltages]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

#create a combined figure of images and voltages using FuncAnimation
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))
ax1.axis("off")
ax1.set_title("Input Frames")
ax2.set_title("Voltages")
ax3.set_title("Position")
ax4.set_title("Reward")
im1 = ax1.imshow(frames[0], animated=True)
im2 = ax2.plot(voltages[0])[0]
im3 = ax3.plot(poses[0][0], poses[0][1], 'ro')[0]
im4 = ax4.plot(rewards[0])[0]
#set plot colors
def updatefig(i):
    im1.set_array(frames[i])
    ax2.clear()
    ax2.set_title("Voltages")
    ax2.set_xlabel("Timestep (1/20 s)")
    ax2.set_ylabel("Voltage (V)")
    ax2.set_ylim(-12, 12)
    #show legend
    ax2.plot([i[0] for i in voltages[:i]], label="Left")
    ax2.plot([i[1] for i in voltages[:i]], label="Right")
    ax2.legend()

    ax3.clear()
    ax3.plot([i[0] for i in poses[:i]], [i[1] for i in poses[:i]], 'ro')
    ax3.set_title("Position")
    ax3.set_xlabel("X (m)")
    ax3.set_ylabel("Y (m)")
    ax3.set_xlim(-3, 12)
    ax3.set_ylim(-3, 5)

    ax4.clear()
    ax4.plot(rewards[:i])
    ax4.set_title("Reward")
    ax4.set_xlabel("Timestep (1/20 s)")
    ax4.set_ylabel("Reward")
    #plot target
    ax3.plot([10], [0], 'go')

    return im1, im2
ani = animation.FuncAnimation(fig, updatefig, frames=len(frames), interval=70, blit=True,
                                repeat_delay=1000)

HTML(ani.to_jshtml())

In [ ]:
#animate x and y poses over time
fig, ax = plt.subplots()
x, y = zip(*poses)
#add to animation
#equal aspect ratio
ax.scatter(x, [-i for i in y])
ax.set_title('x and y positions over time')

#add target pos in red
ax.scatter([env.targetPose[0]], [env.targetPose[1]], color='red')

#equal aspect ratio
ax.set_aspect('equal', 'box')
plt.show()

#plot reward over time
fig, ax = plt.subplots()
ax.plot(rewards)
ax.set_xlabel('timestep')
ax.set_ylabel('reward')
plt.show()

In [ ]:
#show voltage / current graph in a 2x1 grid
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(voltages)
ax1.set_title('Voltage')
ax1.set_ylabel('Volts')
ax1.set_xlabel('Time (sec)')
#stop scientific notation
ax1.ticklabel_format(useOffset=False)
plt.subplots_adjust(hspace=0.5)

ax2.plot(currents)
ax2.set_title('Current')
ax2.set_ylabel('Amps')
ax2.set_xlabel('Time (sec)')
